# Exercise #13

## Task 1

 Create one pandas dataframe that combines all the data scraped from May 22, 2022 together. Drop rows with missing job titles and/or job descriptions. Use `spacy` to tokenize all the job titles included in the cleaned dataframe. For each job title, find all the nouns and all the adjectives in the title and get their lowercased lemmatized form. Use the reformatted nouns to construct a vocabulary set for this dataframe. How many unique nouns are there? Construct another vocabulary set using the reformatted adjectives. How many unique adjectives are there? What kind of different information do the nouns versus the adjectives reveal about the specific job? 

### Preparation + Getting the Dataset


In [1]:
### Preparation
import os
import numpy as np
import pandas as pd
import spacy
import re
import glob
os.chdir('/Users/[editted]/Desktop/SOCDATA/Exercise_13/indeed_scraped_data/job_info_data/')
nlp = spacy.load('en_core_web_sm')
###Data set
extension = 'csv'
all_filenames = [i for i in glob.glob('parsehub_5222022_*.{}'.format(extension))]
all_filenames
combined_csv_info = pd.concat([pd.read_csv(f) for f in all_filenames ])
# combined_csv_info

###Data Cleaning: Drop NAs
combined_csv_info_new = combined_csv_info.dropna(subset=['lnks_job_title','lnks_job_description'])
combined_csv_info_new
combined_csv_info_new.to_csv('job_data.csv',index=False)

combined_csv_info_new.shape

(4803, 6)

In [2]:
job_df = pd.read_csv('job_data.csv')

In [4]:
job_title_info = []
for item in range(len(job_df)):
    job_title_info.append(job_df.loc[item, 'lnks_job_title'])


token_list = []
for item in job_title_info:
    for token in nlp(item):
        if token.is_alpha:
            token_list.append(token.lemma_.lower())
noun_list = list(set(token_list))
len(token_list) # total tokens

19832

In [5]:
doc = nlp(str(token_list))
noun_list = []
for chunk in doc.noun_chunks:
    noun_list.append(chunk)

#noun_list

for element in noun_list:
    if element in noun_list:
        noun_list.remove(element)

len(noun_list)

6418

In [6]:
adj_lst = []
for chunk in doc:
    if chunk.pos_ == 'ADJ':
        adj = chunk.text
        adj_lst.append(adj)


for element in adj_lst:
    if element in adj_lst:
        adj_lst.remove(element)
len(adj_lst)


966

From the analysis, there are 6418 unique nouns and only 966 unique adjectives.

## Task #2

 Choose the first job title in your dataframe as the primary string. Use one-hot encoding as the word embedding method and find jobs in your cleaned dataframe that have similar nouns in the title as your primary string. 

In [55]:
import numpy as np
from tqdm import tqdm

vocab_with_freq = dict()
# job_df = job_df.dropna(subset=['lnks_job_title']).reset_index()

for i in tqdm(range(len(job_df))):
    job_title = job_df.loc[i, 'lnks_job_title'] #Looping through all the rows
    for token in doc.noun_chunks:
        vocab_with_freq[token.lemma_.lower()] = vocab_with_freq.get(token.lemma_.lower(), 0) + 1

vocabulary_list = list(vocab_with_freq.keys())
# print('there are', len(vocabulary_list), 'unique words among indeed job titles')
# print(one_hot_encoding)

100%|███████████████████████████████████████| 4803/4803 [05:09<00:00, 15.53it/s]


In [61]:
for i in tqdm(range(len(job_df))):
    job_title = job_df.loc[i, 'lnks_job_title'] # take one example first
    token_indices = []

for token in nlp(job_title):
    if token.is_alpha:
        print(token.lemma_.lower())
        if token.lemma_.lower() in vocabulary_list:
            token_index_in_vocab = vocabulary_list.index(token.lemma_.lower())
            token_indices.append(token_index_in_vocab)
        else:
            print('token not recognized')

one_hot_encoding = np.zeros(len(vocabulary_list))
for token_index in token_indices:
    one_hot_encoding[token_index] = 1
# print(one_hot_encoding)

100%|████████████████████████████████████| 4803/4803 [00:00<00:00, 96472.73it/s]


grants
token not recognized
manager


In [63]:
one_hot_encodings = []

for i in tqdm(range(len(job_df))):
    job_title = job_df.loc[i, 'lnks_job_title'] # take one example first
    token_indices = []

    for token in nlp(job_title):
        if token.is_alpha and token.lemma_.lower() in vocabulary_list:
            token_index_in_vocab = vocabulary_list.index(token.lemma_.lower())
            token_indices.append(token_index_in_vocab)
        
#         if token.is_alpha:
#             token_index_in_vocab = vocabulary_list.index(token.lemma_.lower())
#             token_indices.append(token_index_in_vocab)


    one_hot_encoding = np.zeros(len(vocabulary_list))
    for token_index in token_indices:
        one_hot_encoding[token_index] = 1

    one_hot_encodings.append(one_hot_encoding)

100%|██████████████████████████████████████| 4803/4803 [00:15<00:00, 313.62it/s]


In [70]:
from scipy.spatial.distance import cosine
title_a = job_df.loc[0, 'lnks_job_title']
similarity_values = []
for i in tqdm(range(1, len(job_df))):
    similarity_value = 1 - cosine(one_hot_encodings[0], one_hot_encodings[i])
    similarity_values.append(similarity_value)
print(similarity_values)

  0%|                                                  | 0/4802 [00:00<?, ?it/s]/Users/zhenjieyu/opt/miniconda3/envs/compsoc/lib/python3.8/site-packages/scipy/spatial/distance.py:620: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)
100%|████████████████████████████████████| 4802/4802 [00:00<00:00, 17769.18it/s]

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.28867513459481287, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.408248290463863, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.33333333333333326, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5773502691896258, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.408248290463863, 0.0, 0.0, 0.0, 0.408248290463863, 0.0, 0.0, 0.0, 0.408248290463863, 0.2581988897471612, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.28867513459481287, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1, 0.408248290463863, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1, 0.0, 0.28867513459481287, 0.0, 0.0, 0.0, 0.333333333333333

In [65]:
similar_df = pd.DataFrame(columns=['job_title', 'similarity_value_with_one_hot'])
similar_df['job_title'] = job_df.loc[1:, 'lnks_job_title']
similar_df['similarity_value_with_one_hot'] = similarity_values
similar_df.sort_values(by='similarity_value_with_one_hot', ascending=False)


,job_title,similarity_value_with_one_hot
3266,LPN,1.0
2147,Cook,1.0
471,RN Neuro/Neuro Surg H8,1.0
1944,Maintenance Controller (A&P) Technician,1.0
3354,Cook,1.0
...,...,...
1645,Water Resources Operator III - Treatment,0.0
1644,Vice President for University Advancement/Univ...,0.0
1643,Meat and Produce Associate,0.0
1642,Dietary Cook- Day/Evening Shift,0.0


### Task #3

Task 3. Use spacy's word vector to do Task 2. Compare the results. 

In [69]:
spacy_encoding = np.array([float(0) for i in range(96)])
token_count = 0

for i in tqdm(range(len(job_df))):
    job_title = job_df.loc[i, 'lnks_job_title']

    for token in nlp(job_title): 
        token_count += 1
        spacy_encoding += np.array(token.vector)
spacy_encoding /= token_count

spacy_encoding

100%|██████████████████████████████████████| 4803/4803 [00:14<00:00, 325.09it/s]


array([ 0.78727029,  0.09429042, -0.0291904 , -0.30666948, -0.01297145,
        0.27987623, -0.66898865,  0.27417195,  0.15756935, -0.27501647,
        0.79000235, -0.29017483,  0.03133199, -0.15388452, -0.10432751,
       -0.21903941,  0.70736634,  0.36688578,  0.26753022,  0.06733849,
       -0.43341728, -0.59232369, -0.20853957,  0.17640644,  0.596861  ,
        0.13703578, -0.36934621,  0.49985243, -0.13412877,  0.71825226,
       -0.15560199,  0.16054758, -0.48123362, -0.37997181,  0.65542782,
       -0.06855982, -0.02637823,  0.29200212, -0.44460662, -0.06709033,
       -0.38061569,  0.11697888, -0.58253866, -0.72070167,  0.26194455,
       -0.99469798,  0.36213415,  0.71141677,  0.22327734, -0.58510638,
        0.25521811, -0.03231519, -0.0020189 , -1.15522798, -0.06106655,
        0.49936272,  0.70040709, -0.22699054, -0.58141121, -0.05059551,
       -0.18371161, -0.32731749,  0.53145739, -0.52224701,  0.60769447,
       -0.51141844,  0.07046536,  0.01111806,  0.27128423, -0.46

Analysis: Compared to one hot coding, the words vecctor shows negative correlations between words.